# LOGEO 

### Exploring the top 5,000 words from hundreds of Spanish speaking cities

In this notebook, we are going to explore a Spanish language dataset and do many interesting things with it. 

When [Google Dataset Search](https://toolbox.google.com/datasetsearch/search?query=spanish%20twitter&docid=KjIKji0L9hTF2vQAAAAAAA%3D%3D) came out, I started looking for datasets in Spanish, to practice programming and data analysis. I found a Twitter dataset and started working with it. 

This dataset is amazing. It contains the top 5000 frequent Spanish words in Twitter for hundreds of cities in the Spanish-speaking world! 

It was created by the [Instituto Caro y Cuervo](https://www.caroycuervo.gov.co/), (a linguistics Institute from Colombia) and made available at the [Open Data Website](https://www.datos.gov.co/) of the Colombian Government.

According to them, 

> More than *250 million tweets* in Spanish from 331 Spanish-speaking cities in Latin America, Spain and the United States were compiled from Twitter. The reported data correspond to the years 2009 to 2016.



Ok, let's get to work. First, let's download the dataset at https://www.datos.gov.co/Ciencia-Tecnolog-a-e-Innovaci-n/The-top-5000-frequent-Spanish-words-in-Twitter-for/nmid-inr9. 

The dataset is a .csv file. We are going to read it using [Pandas](https://pandas.pydata.org/), a great Python library for data analysis. 

In [ ]:
import pandas as pd

df = pd.read_csv('The_top-5000_frequent_Spanish_words_in_Twitter_for_331_cities_in_the_Spanish-speaking_world.csv')

One of the first things we can do is call `head()` to see the first rows. Let's inspect the dataset.

In [ ]:
df.head()

Here, we see that the columns correspond to the cities: we have one column for the words and one column for the word frequency of each city's words. 



By querying the shape, we see that the data set has 5000 rows and 585 columns. 

In [ ]:
df.shape

Although this is a dataset about cities, it does not contain geographical information, such as the location of the cities. 

By consulting other datasets and also by looking up manually the latitude and longitude of the cities, I enhanced the dataset with geographical information. You can download the enhanced dataset [here](https://drive.google.com/file/d/1gHd6N12E1YqS4GMG8HbLFYQsUfGVVXpq/view?usp=sharing). 

In [ ]:
df = pd.read_csv('The_top-5000_frequent_Spanish_words_in_Twitter_for_331_cities_in_the_Spanish-speaking_world_2.csv')

In [ ]:
df.head()

How many countries and cities are included in the dataset? Let's find out:

In [ ]:
from collections import Counter

cnt = Counter()

for country in df['Country'].tolist():
     cnt[country] += 1

print(cnt)

Great! We have 19 countries and we can see how many cities are included for each country. Mexico has the greater number of cities in this dataset, with 74 cities, while Cuba has just one city. Let's keep this in mind and let's go on.

Let's see all the cities included in the data set

In [ ]:
for item in list(zip(df['Country'],df['city_ascii'])):
    print (item[0]+', '+item[1])

At this point, we could also visualize the cities in the dataset on a map. We will use [Folium](https://github.com/python-visualization/folium), a flexible Python library that allows us to make maps easily.

In [ ]:
import folium

m = folium.Map(tiles= 'cartodbpositron')  
m

Let's add information to our map. We are going to zip 3 columns from our dataframe: `city_ascii`, (the city name), `lat` and `lng` (the latitude and longitude of each city)

In [ ]:
cities = list(zip(df['city_ascii'],df['lat'],df['lng']))

Let's see the first element:

In [ ]:
cities[0]

Now, we can add all the cities to our map. We pass the latitude and longitude (`city[1]` and `city[2]` to `location`. We also pass the city name (`city[0]`) to the `popup`, to display the city's name on the Marker. 

In [ ]:
for city in cities:
    folium.Marker(
    popup=city[0],
    location=[city[1],city[2]]
    ).add_to(m)
m

Nice! We get a glance at all the cities included in our dataset! Click on the markers to see the city's name. 

Spain and its islands are included and allmost all of the Latinamerican countries. We have even Spanish speaking cities from the United States. The dataset does not include information about Venezuela and Uruguay, though. 

# Let's dive into text processing

Let's put aside the map for now and let's analyze the words. We are going to use [Gensim](https://radimrehurek.com/gensim/), a Python library for Natural Language Processing.

In order to analyze the words, we need to convert them to vectors. 

In [ ]:
df['lines']

If you have inspected the data set, you see that we have all the country words in one column, joined by a comma (,). Let's create a new column and split this "lines" into tokens.

In [ ]:
from gensim import corpora
df['tokens'] = df['lines'].str.split(',')

In [ ]:
df['tokens']

Let's create a dictionary, with all the *unique* tokens. 

In [ ]:
dictionary = corpora.Dictionary(df['tokens'])

By calling "print" on the dictionary, we see that we have 62314 unique tokens:

In [ ]:
print(dictionary)

Now, let's actually convert the tokens to vectors:

> The function doc2bow() simply counts the number of occurrences of each distinct word, converts the word to its integer word id and returns the result as a sparse vector.

In [ ]:
df['bow'] = df['tokens'].apply(lambda x: dictionary.doc2bow(x))

In [ ]:
df['bow']

Now, let's transform the dataset from one vector representation into another.

According to Gensim's "Topics and Transformations" tutorial:

>This process serves two goals:
>
>To bring out hidden structure in the corpus, discover relationships between words and use them to describe the documents in a new and (hopefully) more semantic way.
To make the document representation more compact. This both improves efficiency (new representation consumes less resources) and efficacy (marginal data trends are ignored, noise-reduction).



# TFIDF

What is tf-idf? 

> In information retrieval, tf–idf or TFIDF, short for term frequency–inverse document frequency, is a numerical statistic that is intended to reflect how important a word is to a document in a collection or corpus.[1] It is often used as a weighting factor in searches of information retrieval, text mining, and user modeling. The tf–idf value increases proportionally to the number of times a word appears in the document and is offset by the number of documents in the corpus that contain the word, which helps to adjust for the fact that some words appear more frequently in general. Tf–idf is one of the most popular term-weighting schemes today; 83% of text-based recommender systems in digital libraries use tf–idf.[2]

Creating a transformation

The transformations are standard Python objects, typically initialized by means of a training corpus:

In [ ]:
from gensim import models
tfidf = models.TfidfModel(df['bow']) 

> From now on, tfidf is treated as a read-only object that can be used to convert any vector from the old representation (bag-of-words integer counts) to the new representation (TfIdf real-valued weights):

Let's apply this transformation to our bag of words column and create a new tfidf column

In [ ]:
df['tfidf'] = df['bow'].apply(lambda x: tfidf[x]) 

#  Similarity
A common reason for this transformations is to determine similarity between pairs of documents, or the similarity between a specific document and a set of other documents (such as a user query vs. indexed documents).

Latent Semantic Indexing, LSI (or sometimes LSA) transforms documents from either bag-of-words or (preferrably) TfIdf-weighted space into a latent space of a lower dimensionality. Let's transform our tf-idf vectors into LSI vectors:

In [ ]:
lsi = models.LsiModel(df['tfidf'],num_topics=200)

To prepare for similarity queries, we need to enter all documents which we want to compare against subsequent queries:

In [ ]:
from gensim import similarities
index = similarities.MatrixSimilarity(lsi[df['tfidf']]) 

# Query

Let's use the lyrics of a Colombian rap group as a query example:

In [ ]:
query = 'Somos pacífico, estamos unidos Nos une la región La pinta, la raza y el don del sabor Somos pacífico, estamos unidos Nos une la región La pinta, la raza y el don del sabor  Ok! si por si acaso usted no conoce En el pacífico hay de todo para que goce Cantadores, colores, buenos sabores Y muchos santos para que adores Es toda una conexión Con un corrillo chocó, valle, cauca Y mis paisanos de nariño Todo este repertorio me produce orgullo Y si somos tantos Porque estamos tan al cucho (en la esquina) Bueno, dejemos ese punto a un lado Hay gente trabajando pero son contados Allá rastrillan, hablan jerguiados Te preguntan si no has janguiado (hanging out) Si estas queda’o Si lo has copiado, lo has vacilado Si dejaste al que está malo o te lo has rumbeado Hay mucha calentura en buenaventura Y si sos chocoano sos arrecho por cultura, ey!  Somos pacífico, estamos unidos Nos une la región La pinta, la raza y el don del sabor Somos pacífico, estamos unidos Nos une la región La pinta, la raza y el don del sabor Unidos por siempre, por la sangre, el color Y hasta por la tierra No hay quien se me pierda Con un vínculo familiar que aterra Característico en muchos de nosotros Que nos reconozcan por la mamá Y hasta por los rostros Étnicos, estilos que entre todos se ven La forma de caminar  El cabello y hasta por la piel Y dime quién me va a decir que no Escucho hablar de san pacho Mi patrono allá en quibdo, ey! Donde se ven un pico y juran que fue un beso Donde el manjar al desayuno es el plátano con queso Y eso que no te he hablado de buenaventura Donde se baila el currulao, salsa poco pega’o Puerto fiel al pescado Negras grandes con gran tumba’o Donde se baila aguabajo y pasillo  En el lado del río (ritmo folclórico) Con mis prietillos Somos pacífico, estamos unidos Nos une la región La pinta, la raza y el don del sabor Somos pacífico, estamos unidos Nos une la región La pinta, la raza y el don del sabor Es del pacífico, guapi, timbiquí, tumaco El bordo cauca Seguimos aquí con la herencia africana Más fuerte que antes Llevando el legado a todas partes De forma constante Expresándonos a través de lo cultural Música, artes plástica, danza en general Acento golpia’o al hablar El 1, 2,3 al bailar Después de eso seguro hay muchísimo más Este es pacífico colombiano Una raza un sector Lleno de hermanas y hermanos Con nuestra bámbara y con el caché (bendición, buen espíritu) Venga y lo ve usted mismo Pa vé como es, y eh! Piense en lo que se puede perder, y eh! Pura calentura y yenyeré, y eh!'.split()

We need to convert our query to the same vector space of our indexed corpus. First, let's transform it to bag of words:

In [ ]:
query_bow = dictionary.doc2bow(query)

In [ ]:
query_bow

Now, let's transform it to lsi:

In [ ]:
query_lsi = lsi[query_bow] 

Let's find out the similarity scores of this query against all our cities in the corpus:

In [ ]:
sim_scores = index[query_lsi]

In [ ]:
df['sim_score'] = sim_scores

In [ ]:
df = df.dropna()

Let's sort our dataframe by similarity score and keep the top 10 cities

In [ ]:
df = df.sort_values(by=['sim_score'], ascending=False)[:10]

Let's finally see which cities are the most similar to our query!

In [ ]:
similar_cities = list(zip(df['city_ascii'],df['lat'],df['lng'],df['sim_score']))
for city in similar_cities:
    print(city)

This is nice! Our similarity model detected correctly that our query (a rap song from Colombia) is indeed from Colombia! We got a list with the most similar cities and their similarity scores

Finally, let's see the results on our map:

In [ ]:
m = folium.Map(tiles= 'cartodbpositron')  

for city in similar_cities:
    folium.Marker(
    popup=city[0],
    location=[city[1],city[2]]
    ).add_to(m)

m.fit_bounds(m.get_bounds())

m